In [1]:
import pandas as pd
import folium
import os
from branca.colormap import linear

Importing and manipulating data

In [2]:
seattle_zips = [98125, 98133, 98177, 98117, 98107, 98103, 98115, 98105,
                98112, 98102, 98109, 98119, 98199, 98121, 98101, 98104,
                98122, 98144, 98134, 98108, 98118, 98106, 98126, 98116, 
                98136]

zillow_columns = ['RegionName', 'City', 'State', '2018-09']

In [3]:
def df_slicer(df, region_list, region_column_str, columns_list):
    df = df[df[region_column_str].isin(region_list)]
    df = df[columns_list]
    df[region_column_str] = df[region_column_str].astype(str)
    print(df)
    return df

Importing Studio Data

In [4]:
studio_rental_path = os.path.join('..', 'data', 'Zip_MedianRentalPrice_Studio.csv')
studio_rental = pd.read_csv(studio_rental_path)

In [5]:
studio_rental = df_slicer(studio_rental, seattle_zips, 'RegionName', zillow_columns)

    RegionName     City State  2018-09
72       98103  Seattle    WA   1375.0
109      98115  Seattle    WA   1443.0
132      98122  Seattle    WA   1565.0
222      98109  Seattle    WA   1850.0
291      98105  Seattle    WA   1100.0
352      98102  Seattle    WA   1448.0
462      98107  Seattle    WA   1395.0
512      98121  Seattle    WA   1800.0
670      98101  Seattle    WA   1937.0
769      98104  Seattle    WA   1650.0


In [6]:
studio_rental_dict = studio_rental.set_index('RegionName')['2018-09']

In [7]:
type(studio_rental_dict['98104'])

numpy.float64

Importing One Bedroom Zillow Rental Price Data

In [8]:
onebr_rental_path = os.path.join('..', 'data', 'Zip_MedianRentalPrice_1Bedroom.csv')

onebr_rental = pd.read_csv(onebr_rental_path)

In [9]:
onebr_rental = df_slicer(onebr_rental, seattle_zips, 'RegionName', zillow_columns)

     RegionName       City State  2018-09
121       98103    Seattle    WA   1695.0
195       98115    Seattle    WA   1724.5
223       98133  Shoreline    WA   1446.0
232       98122    Seattle    WA   2055.0
359       98109    Seattle    WA   2350.0
367       98125    Seattle    WA   1495.0
457       98105    Seattle    WA   1465.0
465       98118    Seattle    WA   1830.0
556       98102    Seattle    WA   1695.0
686       98107    Seattle    WA   1801.5
691       98117    Seattle    WA   1595.0
743       98121    Seattle    WA   2620.0
757       98116    Seattle    WA   1801.0
835       98119    Seattle    WA   1840.0
964       98101    Seattle    WA   2475.0
968       98112    Seattle    WA   1762.5
1009      98126    Seattle    WA   1604.0
1109      98104    Seattle    WA   1999.0


In [10]:
onebr_rental_dict = onebr_rental.set_index('RegionName')['2018-09']

In [11]:
colormap = linear.YlOrRd_09.scale(1050, 2650)
colormap

Initializing a map

In [12]:
zip_codes = os.path.join('..', 'data', 'wa_washington_zip_codes_geo.min.json')

In [13]:
m = folium.Map(location=[47.6553, -122.3035], zoom_start=12, min_zoom=10)
m.add_child(colormap)

## Studio Rental Price Layer

In [14]:
def style_function(feature):
    if feature['properties']['ZCTA5CE10'] in studio_rental_dict: 
        return {
        'fillOpacity' : 0.5,
        'weight' : 1,
        'fillColor': colormap(studio_rental_dict[feature['properties']['ZCTA5CE10']]),
        'color' : 'Black',
        }
    else:
        return {
            'fillColor': 'none',
            'weight' : 0,
            'line_opacity' : 0.1,
            'color': 'Black',
        }

studio_geojson = folium.GeoJson(
    zip_codes,
    name='Average Monthly Rent for a Studio Apartment',
    style_function=style_function,
)

## One Bedroom Rental Price Layer

In [15]:
def style_function_2(feature):
    if feature['properties']['ZCTA5CE10'] in onebr_rental_dict: 
        return {
        'fillOpacity' : 0.5,
        'weight' : 1,
        'fillColor': colormap(onebr_rental_dict[feature['properties']['ZCTA5CE10']]),
        'color' : 'Black',
        }
    else:
        return {
            'fillColor': 'none',
            'weight' : 0,
            'line_opacity' : 0.1,
            'color': 'Black',
        }

onebr_geojson = folium.GeoJson(
    zip_codes,
    name='Average Monthly Rent for a One Bedroom Apartment',
    style_function=style_function_2,
)

In [16]:
studio_geojson.add_to(m)
onebr_geojson.add_to(m)

folium.Choropleth(
    geo_data=zip_codes,
    data=studio_rental,
    columns=['RegionName', '2018-09'],
    key_on= 'feature.properties.ZCTA5CE10',
    fill_color='YlOrRd',
    nan_fill_color = 'none',
    line_weight = 0.5,
    name = 'Studio Rental Pricing by Zip Code',
    legend_name = 'Average Monthly Rental Price for a Studio Apartment'
).add_to(m)

folium.Choropleth(
    geo_data=zip_codes,
    data=onebr_rental,
    columns=['RegionName', '2018-09'],
    key_on= 'feature.properties.ZCTA5CE10',
    fill_color='YlOrRd',
    nan_fill_color = 'none',
    line_weight = 0.5,
    name = 'One Bedroom Rental Pricing by Zip Code',
    legend_name = 'Average Monthly Rental Price for a One Bedroom Apartment'
).add_to(m)

In [17]:
#help(folium.GeoJson)

Adding Greenspace Data

In [17]:
greenspace = os.path.join('..', 'data', 'greenspace2.geojson')

In [18]:
greenspace_geojson = folium.GeoJson(
    greenspace,
    name="Greenspace Markers",
    show=False
)

greenspace_geojson.add_to(m)

## Transit Route Data

In [19]:
transit = os.path.join('..', 'data', 'Transit_Routes_for_King_County_Metro__transitroute_line.geojson')

In [20]:
def style_function3(feature):
    return {
        'fillColor': '#ffaf00',
        'line_opacity' : 0.5,
        'color': 'gray',
        'weight': 1.5,
        'dashArray': '5, 5'
    }

def highlight_function(feature):
    return{
        'fillColor': '#ffaf00',
        'line_opacity' : 1,
        'color': 'blue',
        'weight': 3,
    }

    
transit_geojson = folium.GeoJson(
    transit, 
    name="Transit Routes",
    show=False,
    style_function=style_function3,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(['ROUTE_NUM'], aliases=['Route Number'], interactive=True)
)

transit_geojson.add_to(m)

In [21]:
folium.LayerControl().add_to(m)
m.save('GradPads.html')